In [155]:
from math import ceil
import time
import re
import ast
import sys
import urllib
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
from random import randint
from selenium.webdriver.firefox.webdriver import FirefoxProfile
from urllib.request import urlopen, Request
from urllib.error import HTTPError
from urllib.error import URLError
import requests
from itertools import cycle
import string
from header_list import user_agent_list
from proxies_list import proxies_list, proxie_random_pool
from proxie_check import proxie_check

In [156]:
ua = user_agent_list
proxies = proxies_list
prx_pool = proxie_random_pool

In [157]:
#proxie_check(proxies)
len(proxies)

37

In [132]:
def session_creator(proxy, ua, url):
    header = random.sample(ua, 1)[0]
    session = requests.Session()
    session.proxies = {"http": proxy, "https": proxy}
    req = session.get(url, headers=header)
    soup = BeautifulSoup(req.text, 'lxml')
    return soup

In [133]:
zip_codes = pd.read_csv('../Data/zips.csv')
zip_list = zip_codes['zip'].tolist()
zip_list = [num for num in zip_list if str(num) != 'nan']

In [134]:
# rand_zip_list_ap = random.sample(zip_list, len(zip_list))
# ap_prp_count_list = []

In [135]:
# rand_zip_sld = random.sample(zip_list, len(zip_list))
# sold_prp_list = []
# ezl = []

In [136]:
def zip_prop_count(zip_list, proxies, prp_list, ua, ezl):
    proxy = random.sample(proxies, 1)[0]
    print(proxies.index(proxy))
    print(proxy)
    for num in zip_list:
        url = 'https://www.redfin.com/zipcode/' + \
            str(num)+'/filter/property-type=house+condo+townhouse,include=sold-1yr,min-price=100k,min-baths=1,include=sold-1yr'
        try:
            start_time = time.time()
            soup = session_creator(proxy, ua, url)
            #all_count = soup.findAll('span', {'data-rf-test-name': 'homes-for-sale'})
            print(num)
            print(len(zip_list))
            all_count = soup.findAll('div', {'class': 'homes summary'})
            if len(str(all_count)) >= 20:
                print(all_count)
                print(time.time() - start_time)
                ezl.append(num)
                prp_list.append(all_count)
                zip_list.remove(num)
                print(len(zip_list)+len(prp_list))
            else:
                print("Captcha!!!!!")
        except:
            print("Skipping. Connnection error")
            proxies.remove(proxy)
            print(len(proxies))
            return prp_list, zip_list, proxies, ezl
    return prp_list, zip_list, proxies, ezl

#ap_prp_count_list, rand_zip_list, proxies = zip_prop_count(rand_zip_list_ap, proxies, ap_prp_count_list, ua)
#sold_prp_list, rand_zip_sld, proxies, ezl = zip_prop_count(rand_zip_sld, proxies, sold_prp_list, ua, ezl)

In [137]:
# sold_prp_list = [str(num) for num in sold_prp_list]
# sold_prp_list = [num.replace(
#     '[<div class="homes summary" data-rf-test-id="homes-description"><span class="showingText">Showing </span>', '') for num in sold_prp_list]
# sold_prp_list = [num.replace(
#     ' Homes<span class="summarySeparator ">•</span></div>]', '') for num in sold_prp_list]
# sold_prp_list = [num.replace('20 of ', '') for num in sold_prp_list]

In [138]:
def strip_count(lst):
    rx_num_homes = r'\d+\shomes'
    rx_zip = r'\d+\sat'
    rx_median = r'\$\d+\.?\w+'
    median_list = []
    zip_list = []
    num_homes_list = []
    for num in lst:
        num = str(num)
        median_ = re.findall(rx_median, num, re.MULTILINE)
        zip_ = re.findall(rx_zip, num, re.MULTILINE)
        num_homes_ = re.findall(rx_num_homes, num, re.MULTILINE)
        median_list.extend([i for i in median_])
        zip_list.extend([i for i in zip_])
        num_homes_list.extend([i for i in num_homes_])

    median_list = [num.replace('$', '').replace(
        'K', ',000').replace('.', ',') for num in median_list]
    i = 0
    while i < len(median_list):
        if len(median_list[i]) == 2:
            median_list[i] = median_list[i].replace('M', ',000,000')
            i += 1
        elif len(median_list[i]) == 4:
            median_list[i] = median_list[i].replace('M', '00,000')
            i += 1
        elif len(median_list[i]) == 5:
            median_list[i] = median_list[i].replace('M', '0,000')
            i += 1
        else:
            i += 1
    zip_list = [re.findall(r'\d+', num, re.MULTILINE)[0] for num in zip_list]
    num_homes_list = [re.findall(r'\d+', num, re.MULTILINE)[0]
                      for num in num_homes_list]

    df = pd.DataFrame(
        data={'zip': zip_list, 'median_price': median_list, 'num_ap_homes': num_homes_list})
    return df

In [139]:
# sdf = pd.DataFrame({'zip': ezl, 'sld': sold_prp_list})
# apdf = strip_count(ap_prp_count_list)
# apdf.to_csv('../Data/apdf.csv')
# sdf.to_csv('../Data/sdf.csv')

# apdf.index = ap_df['zip']
# sdf.index = sdf['zip']

# sdf['sld'] = (sdf['sld']).astype(int)
# apdf['num_ap_homes'] = (ap_df['num_ap_homes']).astype(int)

# sdf['zip'] = (sdf['zip']).astype(int)
# apdf['zip'] = (ap_df['zip']).astype(int)

# sld_pgs = [ceil(num/20) for num in sdf['sld']]
# ap_pgs = [ceil(num/20) for num in apdf['num_ap_homes']]

# sdf['pages'] = sld_pgs
# apdf['pages'] = ap_pgs

# sdf = sdf.sort_values(by='pages')
# apdf = apdf.sort_values(by='pages')

# sdf_under = sdf[sdf['pages'] <= 18]
# sdf_over = sdf[sdf['pages'] > 18]

In [154]:
ap_url_list = []
for num in zip_list:
    num_pages_count = apdf.loc[apdf['zip'] == num].pages.values[0]
    for i in range(1, num_pages_count+1):
        url = 'https://www.redfin.com/zipcode/'+str(num) + \
            '/filter/property-type=house+condo+townhouse,min-baths=1,status=active+pending+contingent/page-' + \
            str(i)
        ap_url_list.append(url)


ap_url_list = [num.replace(r'/page-1^', '') for num in ap_url_list]
# ap_url_list = [num.replace('contingent0', 'contingent/page-10')
#                for num in ap_url_list]
# ap_url_list = [num.replace('contingent1', 'contingent/page-11')
#                for num in ap_url_list]
# ap_url_list = [num.replace('contingent2', 'contingent/page-12')
#                for num in ap_url_list]
# ap_url_list = [num.replace('contingent3', 'contingent/page-13')
#                for num in ap_url_list]

# main_df = pd.DataFrame(columns=['full_address', 'home_link'])

ap_url_list = sorted(ap_url_list)
print(ap_url_list)

NameError: name 'apdf' is not defined

In [608]:
# sld_under_url_list = []

# for num in sdf_under['zip']:
#     num_pages_count = sdf_under.loc[sdf_under['zip'] == num].pages.values[0]
#     for i in range(1, num_pages_count+1):
#         url = 'https://www.redfin.com/zipcode/'+str(num) + \
#             '/filter/property-type=house+condo+townhouse,min-price=100k,min-baths=1,include=sold-1yr/page-' + \
#             str(i)
#         sld_under_url_list.append(url)


# sld_under_url_list = [num.replace('/page-1', '') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr0', '=sold-1yr/page-10') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr1', '=sold-1yr/page-11') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr2', '=sold-1yr/page-12') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr3', '=sold-1yr/page-13') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr4', '=sold-1yr/page-14') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr5', '=sold-1yr/page-15') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr6', '=sold-1yr/page-16') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr7', '=sold-1yr/page-17') for num in sld_under_url_list]
# sld_under_url_list = [num.replace(
#     '=sold-1yr8', '=sold-1yr/page-18') for num in sld_under_url_list]

# s_main_df = pd.DataFrame(columns=['full_address', 'home_link'])

sld_under_url_list = sorted(sld_under_url_list)
# print(len(sld_under_url_list))

In [609]:
def each_page(proxy, ua, url):
    soup = session_creator(proxy, ua, url)
    start_time = time.time()
    time.sleep(random.uniform(0, 1)*4)
    print(time.time()-start_time)
    full_soup = soup.findAll('a', {'class': 'bottom link-override'})
    full_address = [fas['title'] for fas in full_soup]
    home_link = ['https://www.redfin.com' +
                 str(hls.get('href')) for hls in full_soup]
    df = {'full_address': full_address, 'home_link': home_link}
    return df

In [140]:
def links_for_props(proxies, url_list, main_df, ua):
    proxy = random.sample(proxies, 1)[0]
    print(proxies.index(proxy))
    print(proxy)
    i = 0
    while i < len(url_list):
        url = url_list[i]        
        try:
            start_time = time.time()
            print(len(url_list))
            print(url)
            
            data = each_page(proxy, ua, url)
            df = pd.DataFrame(data)
            eds = {'full_address': [], 'home_link': []}
            if data['full_address'] != eds['full_address']:
                main_df = pd.concat([main_df, df])
                url_list.pop(i)
                print(time.time() - start_time)
            else:
                print('Captcha')
                return url_list, main_df
        except:
            print("Skipping. Connnection error")
            proxies.remove(proxy)
            print(len(proxies))

            return url_list, main_df

    return url_list, main_df

In [611]:
# a = randint(0,3)
# time.sleep(random.uniform(0,1)*((88/13)**.33)*a)
# ap_url_list, main_df = links_for_props(proxies,ap_url_list, main_df, ua)

In [612]:
# main_df.to_csv('../Data/main_df_ap.csv')

In [613]:
# sld_under_url_list = sld_under_url_list[1:]
print(len(sld_under_url_list))

0


In [614]:
# sld_under_url_list, s_main_df = links_for_props(
#     proxies, sld_under_url_list, s_main_df, ua)

In [616]:
list_ = list(s_main_df.full_address.values)
#set([num[-5:] for num in list_])

In [21]:
s_main_df = pd.read_csv('../Data/s_main.csv')
main_df = pd.read_csv('../Data/main_df_ap.csv')

In [572]:
# s_main_df.to_csv('../Data/s_main.csv')

In [ ]:
#     all_home_feats = soup.findAll('div', {'class':"amenities-container"})
#     for num in all_home_feats:
#         print(num.findAll('li'))

#     other_home_feats = soup.findAll('span', {'data-rf-test-id':"propertyDetails"})
#     print(f'other_home_feats = {other_home_feats}')
#     other_home_feats_vals = soup.findAll('span', {'class':"content font-weight-roman"})
#     print(f'other_home_feats_vals = {other_home_feats_vals}')
    #both are in soup.findAll('div', {'class':"keyDetail font-size-base"})

#     home_full_feat_desc = soup.findAll('span', {'class': 'statsLabel'})
#     for desc in home_full_feat_desc:
#         print(f'hffd = {desc}')
#         homefeat_desc.append(str(desc.contents[0]))

#     home_full_feat_info = soup.findAll('span', {'class': 'statsValue'})
#     for info in home_full_feat_info:
#         print(f'info = {info}')
#         homefeat_info.append(str(info.contents[0]))

#     school_grades = soup.findAll('div', {'class': 'nearby-schools-grades'})
#     for grade in school_grades:
#         print(grade.text)

#     school_assigned = soup.findAll(
#         'span', {'class': 'assigned-label zsg-fineprint'})

#     for assi in school_assigned:
#         print(assi.text)

#     school_dist = soup.findAll('div', {'class': 'nearby-schools-distance'})
#     for dist in school_dist:
#         print(dist.text)

#     school_rating = soup.findAll('div', {'class': 'nearby-schools-rating'})
#     for rating in school_rating:
#         print(rating.span.text)

#     school_name_regex = r'school-name notranslate$'
#     school_name = soup.findAll('a', {'class': 'school-name notranslate'})
#     for mane in school_name:
#         print(mane.next_element)

#     homefeat_desc = []
#     homefeat_info = []

#     new_dict = dict(zip(homefeat_desc, homefeat_info))
#     new_dict['address'] = str(home_address_MLS)

# #     new_dict['home_description'] = str(home_description.attrs['content'])

# lots of info:

#     estimate = soup.find('div', {'data-rf-test-name': 'avmValue"'})
#     print(f'estimate = {estimate}')

#     home_hist_src = soup.findAll('td', {'class': 'zsg-sm-hide'})
#     print(home_hist_src)

In [150]:
def each_property(url, hdr, proxy):
    soup = session_creator(proxy, ua, url)

    price_regex = r'\$\S+\s+'
    bed_regex = r'\d+\S?\d?\d?Bed'
    bath_regex = r'\d+\S?\d?\d?Bath'
    size_regex = r'\d+\S?\d?\d?\s?Sq'
    yr_blt_regex = r'Built: \d+'
    status_regex = r'Status: \w+'

    home_address_MLS = soup.title.string

    all_top = soup.findAll('div', {'class': 'HomeInfo inline-block'})
    all_top_list = []
    for num in all_top:
        a = num.text
        all_top_list.extend(price_regex, a)
        all_top_list.extend(bed_regex, a)
        all_top_list.extend(bath_regex, a)
        all_top_list.extend(size_regex, a)
        all_top_list.extend(yr_blt_regex, a)
        all_top_list.extend(status_regex, a)

    home_description = soup.find('p', {'class': 'font-b1'})
    home_desc_text = home_description.span.text

    all_home_feats = soup.findAll('span', {'class': "entryItemContent"})
    feats = [num.text for num in all_home_feats]
    
    prop_hist = soup.findAll('div', {'id': 'propertyHistory-expandable-segment'})
    prop_history = [num.text for num in prop_hist]

    school_info = soup.findAll('div', {'class': "name-and-info"})
    schools = [num.text for num in school_info]
    
    return home_address_MLS, all_top_list, home_desc_text, feats, prop_history, schools

In [153]:
header = random.sample(ua, 1)[0]
proxy = random.sample(proxies, 1)[0]
sold_url_list = s_main_df['home_link'].tolist()
url = random.sample(sold_url_list, 1)[0]
print(proxy)

home_address_MLS, all_top_list, home_desc_text, feats, prop_history, schools = each_property(url, header, proxy)

64.123.99.70:38913


TypeError: extend() takes exactly one argument (2 given)

In [147]:
price_regex = r'\$\S+\s+'
bed_regex = r'\d+\S?\d?\d?Bed'
bath_regex = r'\d+\S?\d?\d?Bath'
size_regex = r'\d+\S?\d?\d?\s?Sq'
yr_blt_regex = r'Built: \d+'
status_regex = r'Status: \w+'

In [148]:
print(ptp)

None
